In [42]:
import sys
sys.path.append("../..")

import dataInterpreter as dt
import enrichmentAnalysis as ea
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'

path = "C:\\Users\\Pedro\\Documents\\BicPAMS\\data\\latecovid\\"
results_path = "C:\\Users\\Pedro\\Documents\\MasterThesis\\Biclustering\\Results\\"

In [5]:
filtered_data_01 = pd.read_csv(path + 'data-p01.csv', index_col = 0, sep = '\t')
filtered_data_02 = pd.read_csv(path + 'data-p02.csv', index_col = 0, sep = '\t')
filtered_data_02_03 = pd.read_csv(path + 'data-p02-03.csv', index_col = 0, sep = '\t')
filtered_data_05 = pd.read_csv(path + 'data-p05.csv', index_col = 0, sep = '\t')

top200 = pd.read_csv(path + 'data-top200.csv', index_col = 0, sep = '\t')
top500 = pd.read_csv(path + 'data-top500.csv', index_col = 0, sep = '\t')
top1000 = pd.read_csv(path + 'data-top1000.csv', index_col = 0, sep = '\t')
top2000 = pd.read_csv(path + 'data-top2000.csv', index_col = 0, sep = '\t')
top5000 = pd.read_csv(path + 'data-top5000.csv', index_col = 0, sep = '\t')

In [7]:
def binary_search_results(data, value, data_selector_lambda = lambda x, i: x[i]):
    start = 0
    end = len(data) - 1
    
    # detect a boundary in an ordered vector
    while start < end:
        i = int((end + start) / 2)
        
        if data_selector_lambda(data, i) < value and data_selector_lambda(data, i + 1) >= value:
            break
        
        if data_selector_lambda(data, i) >= value:
            # go left
            end = i - 1
        else:
            # go right
            start = i + 1
            
    i = int((end + start) / 2)
    return i + 1

In [43]:
import re

def import_bicPAM(filename):
    file = open(filename, 'r')
    
    # get number of biclusters found
    num_biclusters = int(re.search("[0-9]+", file.readline()).group())
    
    rx = re.compile('[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?', re.VERBOSE)
    
    # read summary lines
    for i in range(num_biclusters):
        bic_info = file.readline() #.split(' ')
        #print(bic_info[3][3:-1].split(','))
    
    file.readline()
    
    result = []
    
    for i in range(num_biclusters):
        bic_info = file.readline().split(' ')
        size = eval(bic_info[2])
        conditions = bic_info[3][3:-1].split(',')
        pvalue = float(bic_info[6].split('=')[1])
        
        genes = []
        for j in range(size[1]):
            genes.append(file.readline().split('\t')[0])
            
        #enriched_terms = ea.getEnrichment(genes, 'GO_Biological_Process_2018')['GO_Biological_Process_2018']
        
        bic = {'size': size, 'conditions': conditions, 'genes': genes, 'pvalue': pvalue}#, 'enriched_terms': enriched_terms}
        result.append(bic)
        
        file.readline()
        file.readline()
    
    return result
    

In [49]:
res = import_bicPAM(results_path + 'result-p05-bicPAM.txt')
res

[{'size': (3, 4463),
  'conditions': ['Series2_A549_SARS-CoV-2_1',
   'Series5_A549_SARS-CoV-2_3',
   'Series15_COVID19Lung_1'],
  'genes': ['SAMD11',
   'NOC2L',
   'AGRN',
   'SDF4',
   'UBE2J2',
   'DVL1',
   'CCNL2',
   'SSU72',
   'SLC35E2B',
   'NADK',
   'RER1',
   'CEP104',
   'RPL22',
   'ACOT7',
   'DNAJC11',
   'VAMP3',
   'PARK7',
   'ERRFI1',
   'RERE',
   'H6PD',
   'SPSB1',
   'KIF1B',
   'DFFA',
   'TARDBP',
   'EXOSC10',
   'PLOD1',
   'MFN2',
   'VPS13D',
   'DHRS3',
   'TMEM51',
   'EFHD2',
   'PLEKHM2',
   'FBLIM1',
   'SPEN',
   'EPHA2',
   'NECAP2',
   'NBPF1',
   'SDHB',
   'RCC2',
   'IFFO2',
   'UBR4',
   'EMC1',
   'MRTO4',
   'MUL1',
   'DDOST',
   'HP1BP3',
   'EIF4G3',
   'ECE1',
   'USP48',
   'ZBTB40',
   'EPHB2',
   'KDM1A',
   'LUZP1',
   'HTR1D',
   'HNRNPR',
   'GALE',
   'SRSF10',
   'SRRM1',
   'SYF2',
   'ARID1A',
   'SFN',
   'NUDC',
   'TRNP1',
   'WASF2',
   'PPP1R8',
   'RPA2',
   'DNAJC8',
   'PHACTR4',
   'SNHG3',
   'RCC1',
   'YTHDF2',
   '

In [50]:
for i in range(len(res)):
    print(i, res[i]['conditions'])

0 ['Series2_A549_SARS-CoV-2_1', 'Series5_A549_SARS-CoV-2_3', 'Series15_COVID19Lung_1']
1 ['Series9_NHBE_Mock_1', 'Series9_NHBE_Mock_2', 'Series15_COVID19Lung_1']
2 ['Series1_NHBE_SARS-CoV-2_3', 'Series9_NHBE_IAV_1', 'Series5_A549_SARS-CoV-2_3']
3 ['Series2_A549_Mock_1', 'Series5_A549_Mock_1', 'Series2_A549_SARS-CoV-2_1']
4 ['Series4_A549_Mock_1', 'Series8_A549_Mock_2', 'Series15_COVID19Lung_1']
5 ['Series4_A549_Mock_2', 'Series4_A549_IAV_2', 'Series15_COVID19Lung_1']
6 ['Series8_A549_Mock_1', 'Series3_A549_RSV_2', 'Series15_COVID19Lung_1']
7 ['Series5_A549_SARS-CoV-2_1', 'Series5_A549_SARS-CoV-2_2', 'Series5_A549_SARS-CoV-2_3']
8 ['Series9_NHBE_Mock_1', 'Series9_NHBE_Mock_2', 'Series15_COVID19Lung_1']
9 ['Series9_NHBE_Mock_3', 'Series9_NHBE_IAV_2', 'Series15_COVID19Lung_1']
10 ['Series1_NHBE_SARS-CoV-2_3', 'Series2_A549_SARS-CoV-2_1', 'Series2_A549_SARS-CoV-2_2']
11 ['Series1_NHBE_SARS-CoV-2_3', 'Series7_Calu3_Mock_3', 'Series15_COVID19Lung_1']
12 ['Series9_NHBE_IAV_1', 'Series2_A549_S

In [52]:
for gene in res[7]['genes']:
    print(gene)

NOC2L
AGRN
SDF4
DVL1
CCNL2
SSU72
SLC35E2B
GNB1
RER1
RPL22
ACOT7
DNAJC11
VAMP3
PARK7
ERRFI1
RERE
ENO1
H6PD
SPSB1
KIF1B
TARDBP
EXOSC10
PLOD1
MFN2
VPS13D
DHRS3
TMEM51
EFHD2
PLEKHM2
FBLIM1
SPEN
EPHA2
NBPF1
SDHB
RCC2
IFFO2
UBR4
EMC1
MRTO4
CAPZB
DDOST
HP1BP3
EIF4G3
ECE1
USP48
CDC42
ZBTB40
EPHB2
KDM1A
LUZP1
HTR1D
HNRNPR
RPL11
GALE
SRRM1
CLIC4
SYF2
TMEM50A
ARID1A
SFN
NUDC
TRNP1
WASF2
PPP1R8
DNAJC8
PHACTR4
SNHG3
RCC1
YTHDF2
SRSF4
PUM1
PTP4A2
KHDRBS1
KPNA6
TXLNA
EIF3I
HDAC1
MARCKSL1
BSDC1
RBBP4
YARS
RNF19B
AK2
PHC2
SFPQ
ZMYM4
KIAA0319L
PSMB2
TRAPPC3
MAP7D1
THRAP3
ZC3H12A
MEAF6
GNL2
SF3A3
RRAGC
AKIRIN1
MACF1
PABPC4
PPIE
CAP1
PPT1
ZMPSTE24
NFYC
CITED4
CTPS1
FOXJ3
YBX1
SLC2A1
EBNA1BP2
ELOVL1
MED8
PTPRF
KDM4A
RNF220
KIF2C
RPS8
PRDX1
AKR1A1
NASP
GPBP1L1
POMGNT1
UQCRH
ATPAF1
EFCAB14
PDZK1IP1
CMPK1
FAF1
EPS15
OSBPL9
RAB3B
TXNDC12
BTF3L4
CC2D1B
LRP8
LRRC42
TMEM59
MRPL37
DHCR24
USP24
JUN
USP1
DOCK7
PGM1
ROR1
JAK1
LEPROT
PDE4B
SERBP1
GADD45A
GNG12
SRSF11
ZRANB2
CRYZ
ACADM
RABGGTB
PIGK
ZZZ3
USP33
FUBP1
GNG

In [9]:
def import_bics_biclust(filename):
    file = open(filename, 'r')
    
    # Read line of algorithm parameters
    line = file.readline()
    
    result = []
    
    while line != '':
        # size line, gene line, conditions
        for i in range(3):
            size_text = file.readline().split(' ')
            if len(size_text) != 2: 
                return result
            size = (int(size_text[1]), int(size_text[0]))
            
            genes = file.readline()[:-1].split(' ')
            conditions = file.readline()[:-1].split(' ')
            
            enriched_terms = ea.getEnrichment(list(genes), 'GO_Biological_Process_2018')['GO_Biological_Process_2018']
            
            bic = {'size': size, 'conditions': conditions, 'genes': genes, 'enriched_terms': enriched_terms}
            result.append(bic)
    
    # usually shouldn't reach this
    return result

In [21]:
bicpam_p01[0]['size']

(7, 272)

In [117]:
results = {}

results['bicpam_p01'] = import_bicPAM(results_path + 'result-p01-bicPAM.txt')
results['bicpam_p02'] = import_bicPAM(results_path + 'result-p02-bicPAM.txt')
results['bicpam_p05'] = import_bicPAM(results_path + 'result-p05-bicPAM.txt')
results['bicpam_anova200'] = import_bicPAM(results_path + 'result-top200-bicPAM.txt')
results['bicpam_anova500'] = import_bicPAM(results_path + 'result-top500-bicPAM.txt')
results['bicpam_anova1000'] = import_bicPAM(results_path + 'result-top1000-bicPAM.txt')
results['bicpam_anova2000'] = import_bicPAM(results_path + 'result-top2000-bicPAM.txt')
results['bicpam_anova5000'] = import_bicPAM(results_path + 'result-top5000-bicPAM.txt')

results['cheng_church_p01'] = import_bics_biclust(results_path + 'results-p01-chengAndChurch.txt')
results['cheng_church_p02'] = import_bics_biclust(results_path + 'results-p02-chengAndChurch.txt')
results['cheng_church_p05'] = import_bics_biclust(results_path + 'results-p05-chengAndChurch.txt')
results['cheng_church_anova200'] = import_bics_biclust(results_path + 'results-top200-chengAndChurch.txt')
results['cheng_church_anova500'] = import_bics_biclust(results_path + 'results-top500-chengAndChurch.txt')
results['cheng_church_anova1000'] = import_bics_biclust(results_path + 'results-top1000-chengAndChurch.txt')
results['cheng_church_anova2000'] = import_bics_biclust(results_path + 'results-top2000-chengAndChurch.txt')
results['cheng_church_anova5000'] = import_bics_biclust(results_path + 'results-top5000-chengAndChurch.txt')

results['plaid_p01'] = import_bics_biclust(results_path + 'results-p01-plaid.txt')
results['plaid_p02'] = import_bics_biclust(results_path + 'results-p02-plaid.txt')
results['plaid_p05'] = import_bics_biclust(results_path + 'results-p05-plaid.txt')
results['plaid_anova200'] = import_bics_biclust(results_path + 'results-top200-plaid.txt')
results['plaid_anova500'] = import_bics_biclust(results_path + 'results-top500-plaid.txt')
results['plaid_anova1000'] = import_bics_biclust(results_path + 'results-top1000-plaid.txt')
results['plaid_anova2000'] = import_bics_biclust(results_path + 'results-top2000-plaid.txt')
results['plaid_anova5000'] = import_bics_biclust(results_path + 'results-top5000-plaid.txt')

results['xMotifs_p01'] = import_bics_biclust(results_path + 'results-p01-xMotifs.txt')
results['xMotifs_p02'] = import_bics_biclust(results_path + 'results-p02-xMotifs.txt')
results['xMotifs_p05'] = import_bics_biclust(results_path + 'results-p05-xMotifs.txt')
results['xMotifs_anova200'] = import_bics_biclust(results_path + 'results-top200-xMotifs.txt')
results['xMotifs_anova500'] = import_bics_biclust(results_path + 'results-top500-xMotifs.txt')
results['xMotifs_anova1000'] = import_bics_biclust(results_path + 'results-top1000-xMotifs.txt')
results['xMotifs_anova2000'] = import_bics_biclust(results_path + 'results-top2000-xMotifs.txt')
results['xMotifs_anova5000'] = import_bics_biclust(results_path + 'results-top5000-xMotifs.txt')

Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value
Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlap

In [118]:
import json

#with open('processed_results.txt', 'w') as file:
     #file.write(json.dumps(results)) # use `json.loads` to do the reverse

NameError: name 'json' is not defined

In [4]:
import json

with open('processed_results.txt') as file:
    results = json.load(file)

In [5]:
def prettify_biclust_method(method):
    if method == 'bicpam':
        return 'BicPAM'
    elif method == 'cheng_church':
        return 'Cheng and Church'
    elif method == 'plaid':
        return 'Plaid'
    else:
        return method

def prettify_gene_select_method(method):
    if method.startswith('p'):
        return 'p-value < 0.' + method.rpartition('p')[-1] + ' (Mann-Whitney U test)'
    elif method.startswith('anova'):
        return 'ANOVA, top ' + method.rpartition('anova')[-1] + ' features'

In [10]:
table1 = {'Number of Biclusters': [], 'Average Number of Genes per Bicluster': [], 'Standard Deviation of Genes per Bicluster': [], 'Average Number of Conditions per Bicluster': [], 'Standard Deviation of Conditions per Bicluster': [], 'Average of Enriched Terms per Bicluster': []}
table1_index = []

for method in results:
    method_info = method.rpartition('_')
    
    table1_index.append((prettify_biclust_method(method_info[0]), prettify_gene_select_method(method_info[2])))
    table1['Number of Biclusters'].append(len(results[method]))
    table1['Average Number of Genes per Bicluster'].append(np.mean([x['size'][1] for x in results[method]]))
    table1['Standard Deviation of Genes per Bicluster'].append(np.std([x['size'][1] for x in results[method]]))
    table1['Average Number of Conditions per Bicluster'].append(np.mean([x['size'][0] for x in results[method]]))
    table1['Standard Deviation of Conditions per Bicluster'].append(np.std([x['size'][0] for x in results[method]]))
    table1['Average of Enriched Terms per Bicluster'].append(np.mean([binary_search_results(x['enriched_terms'], 0.01, lambda x, i: x[i][6]) for x in results[method]]))

In [11]:
pd.DataFrame(table1, pd.MultiIndex.from_tuples(table1_index))

Number of Biclusters  \
BicPAM           p-value < 0.01 (Mann-Whitney U test)                    80   
                 p-value < 0.02 (Mann-Whitney U test)                    77   
                 p-value < 0.05 (Mann-Whitney U test)                    79   
                 ANOVA, top 200 features                                  7   
                 ANOVA, top 500 features                                 47   
                 ANOVA, top 1000 features                                20   
                 ANOVA, top 2000 features                                98   
                 ANOVA, top 5000 features                                57   
Cheng and Church p-value < 0.01 (Mann-Whitney U test)                    50   
                 p-value < 0.02 (Mann-Whitney U test)                   100   
                 p-value < 0.05 (Mann-Whitney U test)                   100   
                 ANOVA, top 200 features                                  8   
                 ANOVA, top 500 features                                 28   
                 ANOVA, top 1000 features                                56   
                 ANOVA, top 2000 features                               100   
                 ANOVA, top 5000 features                               100   
Plaid            p-value < 0.01 (Mann-Whitney U test)                    10   
                 p-value < 0.02 (Mann-Whitney U test)                    10   
                 p-value < 0.05 (Mann-Whitney U test)                    10   
                 ANOVA, top 200 features                                  8   
                 ANOVA, top 500 features                                 10   
                 ANOVA, top 1000 features                                10   
                 ANOVA, top 2000 features                                10   
                 ANOVA, top 5000 features                                10   
xMotifs          p-value < 0.01 (Mann-Whitney U test)                    10   
                 p-value < 0.02 (Mann-Whitney U test)                    10   
                 p-value < 0.05 (Mann-Whitney U test)                    10   
                 ANOVA, top 200 features                                  6   
                 ANOVA, top 500 features                                 10   
                 ANOVA, top 1000 features                                10   
                 ANOVA, top 2000 features                                10   
                 ANOVA, top 5000 features                                10   

                                                       Average Number of Genes per Bicluster  \
BicPAM           p-value < 0.01 (Mann-Whitney U test)                             208.025000   
                 p-value < 0.02 (Mann-Whitney U test)                             922.701299   
                 p-value < 0.05 (Mann-Whitney U test)                            3526.658228   
                 ANOVA, top 200 features                                          188.285714   
                 ANOVA, top 500 features                                          380.191489   
                 ANOVA, top 1000 features                                         676.050000   
                 ANOVA, top 2000 features                                         943.867347   
                 ANOVA, top 5000 features                                        2106.175439   
Cheng and Church p-value < 0.01 (Mann-Whitney U test)                              15.600000   
                 p-value < 0.02 (Mann-Whitney U test)                              23.870000   
                 p-value < 0.05 (Mann-Whitney U test)                              55.900000   
                 ANOVA, top 200 features                                           25.000000   
                 ANOVA, top 500 features                                           17.821429   
                 ANOVA, top 1000 features                                          17.857143   
       

In [197]:
results['plaid_p01'][2]

{'size': [18, 94],
 'conditions': ['Series1_NHBE_Mock_1',
  'Series1_NHBE_Mock_2',
  'Series1_NHBE_Mock_3',
  'Series9_NHBE_Mock_1',
  'Series9_NHBE_Mock_2',
  'Series9_NHBE_Mock_3',
  'Series9_NHBE_Mock_4',
  'Series1_NHBE_SARS.CoV.2_1',
  'Series1_NHBE_SARS.CoV.2_2',
  'Series1_NHBE_SARS.CoV.2_3',
  'Series9_NHBE_IAV_1',
  'Series9_NHBE_IAV_2',
  'Series9_NHBE_IAV_3',
  'Series9_NHBE_IAV_4',
  'Series9_NHBE_IAVdNS1_1',
  'Series9_NHBE_IAVdNS1_2',
  'Series9_NHBE_IAVdNS1_3',
  'Series9_NHBE_IAVdNS1_4'],
 'genes': ['LOC100506801',
  'LUZP1',
  'INPP5B',
  'PTPRF',
  'ROR1',
  'CTSS',
  'PFKP',
  'HKDC1',
  'GRID1',
  'HSPA12A',
  'SYT9',
  'DENND5A',
  'NUCB2',
  'RASGRP2',
  'UBASH3B',
  'DUSP16',
  'GPR19',
  'GRIN2B',
  'AMIGO2',
  'C12orf65',
  'STX2',
  'DCLK1',
  'LCP1',
  'HAUS4',
  'AJUBA',
  'SNAPC1',
  'KLC1',
  'UACA',
  'HCN4',
  'RGS11',
  'FOXL1',
  'PITPNA-AS1',
  'TRIM16L',
  'DHRS13',
  'ITGA3',
  'C1QTNF1',
  'PRTN3',
  'CELF5',
  'LINC00662',
  'CD22',
  'ZNF566',
  

In [158]:
results['plaid_p01'][2]['enriched_terms']

{'GO_Biological_Process_2018': [[1,
   'regulation of signal transduction (GO:0009966)',
   0.00011528014909489325,
   7.006408300274641,
   63.535128588717825,
   ['UBASH3B', 'ITGA3', 'AXL', 'HHIP', 'ROR1', 'FOXL1', 'RGS11'],
   0.06985977035150531,
   0,
   0],
  [2,
   'negative regulation of cell communication (GO:0010648)',
   0.0005003052558027592,
   11.911111111111111,
   90.52792408763345,
   ['UBASH3B', 'AXL', 'HHIP', 'ROR1'],
   0.11251701215078214,
   0,
   0],
  [3,
   'negative regulation of signaling (GO:0023057)',
   0.0006331080518577728,
   11.154430379746836,
   82.15092356215999,
   ['UBASH3B', 'AXL', 'HHIP', 'ROR1'],
   0.11251701215078214,
   0,
   0],
  [4,
   'extracellular matrix organization (GO:0030198)',
   0.0007426865488500471,
   6.01803913575214,
   43.361395086500494,
   ['ITGA3', 'COL4A6', 'COL4A5', 'LCP1', 'CTSS', 'MATN3'],
   0.11251701215078214,
   0,
   0],
  [5,
   'positive regulation of cation transmembrane transport (GO:1904064)',
   0.00283885

In [3]:
for gene in results['plaid_p01'][2]['genes']:
    print(gene)

LOC100506801
LUZP1
INPP5B
PTPRF
ROR1
CTSS
PFKP
HKDC1
GRID1
HSPA12A
SYT9
DENND5A
NUCB2
RASGRP2
UBASH3B
DUSP16
GPR19
GRIN2B
AMIGO2
C12orf65
STX2
DCLK1
LCP1
HAUS4
AJUBA
SNAPC1
KLC1
UACA
HCN4
RGS11
FOXL1
PITPNA-AS1
TRIM16L
DHRS13
ITGA3
C1QTNF1
PRTN3
CELF5
LINC00662
CD22
ZNF566
AXL
HSD17B14
ZNF160
PRKCG
PTPRH
ZFP28
IAH1
CYS1
MATN3
FAM161A
ST3GAL5
LIPT1
NPAS2
C2orf49
NAB1
TRIB3
EEF1A2
IFNAR2
IL10RB
STAC
SLC35G2
SHOX2
LIPH
CXCL5
HHIP
MAP1B
SLCO4C1
EFNA5
LOC100268168
STC2
TPBG
AKAP12
ZNF853
BMPER
LINC00525
SEMA3C
GATAD1
TMEM130
LOC254896
NEFL
PNMA2
ASPH
MYBL1
ZNF34
PLIN2
SVEP1
PPP1R26
CRLF2
CLCN4
GSPT2
COL4A6
COL4A5
NLGN4Y


In [12]:
def get_info_from_name(col_name):
    result = {}
    info = col_name.split('_')
    result['Series'] = info[0]
    
    if info[1] == 'COVID19Lung':
        result['Cell Type'] = 'Biopsy'
        result['Condition'] = 'SARS-CoV-2'
        return result
    elif info[1] == 'HealthyLungBiopsy':
        result['Cell Type'] = 'Biopsy'
        result['Condition'] = 'Healthy'
        return result
        
    result['Cell Type'] = info[1]
    
    if info[2] == 'Mock':
        result['Condition'] = 'Healthy'
    else:
        result['Condition'] = info[2].replace('.', '-')
    
    return result

get_info_from_name('Series2_A549_SARS-CoV-2_1')

{'Series': 'Series2', 'Cell Type': 'A549', 'Condition': 'SARS-CoV-2'}

In [13]:
index = {}

cell_types = set()
conditions = set()

for method in results:
    for biclust in results[method]: 
        for cond in biclust['conditions']:
            info = get_info_from_name(cond)
            
            if info['Cell Type'] not in index:
                index[info['Cell Type']] = set()
            
            index[info['Cell Type']].add(info['Condition'])

multiindex = []

for key in index:
    for subkey in index[key]:
        multiindex.append((key, subkey))

In [14]:
def find_in_multiindex(e0, e1, multiindex):
    i = 0
    for e in multiindex:
        if e[0] == e0 and e[1] == e1:
            return i
        
        i += 1

In [15]:
multiindex

[('A549', 'Healthy'),
 ('A549', 'HPIV3'),
 ('A549', 'SARS-CoV-2'),
 ('A549', 'IAV'),
 ('A549', 'RSV'),
 ('NHBE', 'Healthy'),
 ('NHBE', 'IAVdNS1'),
 ('NHBE', 'IAV'),
 ('NHBE', 'SARS-CoV-2'),
 ('Calu3', 'Healthy'),
 ('Calu3', 'SARS-CoV-2'),
 ('Biopsy', 'Healthy'),
 ('Biopsy', 'SARS-CoV-2')]

In [16]:
find_in_multiindex('Calu3', 'Healthy', multiindex)

9

In [26]:
mentions_per_condition = {}

for cond in multiindex:
    if cond[0] not in mentions_per_condition:
        mentions_per_condition[cond[0]] = {}
        
    mentions_per_condition[cond[0]][cond[1]] = 0

for method in results:
    for biclust in results[method]:
        biclust_terms = biclust['enriched_terms']
        conditions = biclust['conditions']
        
        unique_conditions = set()
        
        for cond in conditions:
            info = get_info_from_name(cond)
            unique_conditions.add((info['Cell Type'], info['Condition']))
            
        for cond in unique_conditions:
            mentions_per_condition[cond[0]][cond[1]] += 1
    
mentions_per_condition

{'A549': {'Healthy': 895,
  'HPIV3': 419,
  'SARS-CoV-2': 687,
  'IAV': 376,
  'RSV': 220},
 'NHBE': {'Healthy': 486, 'IAVdNS1': 291, 'IAV': 379, 'SARS-CoV-2': 315},
 'Calu3': {'Healthy': 114, 'SARS-CoV-2': 103},
 'Biopsy': {'Healthy': 61, 'SARS-CoV-2': 174}}

In [34]:
term_index = []
terms = {}

data = []

i = 0

for method in results:
    for biclust in results[method]:
        biclust_terms = biclust['enriched_terms']
        conditions = biclust['conditions']
        
        if 'GO_Biological_Process_2018' in biclust['enriched_terms']:
            biclust_terms = biclust['enriched_terms']['GO_Biological_Process_2018']
        
        for term in biclust_terms:
            if term[6] >= 0.01:
                break

            if term[1] not in terms:
                term_index.append((term[1], 'Occurrences'))
                term_index.append((term[1], 'p-value'))
                terms[term[1]] = i
                data.append([0] * len(multiindex)) # occurrences
                data.append([1] * len(multiindex)) # p-values
                i += 2
            
            
            for cond in conditions:
                info = get_info_from_name(cond)
                
                row_index = terms[term[1]]
                col_index = find_in_multiindex(info['Cell Type'], info['Condition'], multiindex)
                
                data[row_index][col_index] += 1 / mentions_per_condition[info['Cell Type']][info['Condition']]
                
                if data[row_index + 1][col_index] > term[6]:
                    data[row_index + 1][col_index] = term[6]


In [35]:
enriched_dataframe = pd.DataFrame(data, index = pd.MultiIndex.from_tuples(term_index), columns = pd.MultiIndex.from_tuples(multiindex)).sort_index().sort_index(axis = 1)

In [36]:
enriched_dataframe

A549  \
                                                                   HPIV3   
'de novo' posttranslational protein folding (GO... Occurrences  0.004773   
                                                   p-value      0.000030   
5S class rRNA transcription from RNA polymerase... Occurrences  0.000000   
                                                   p-value      1.000000   
7-methylguanosine RNA capping (GO:0009452)         Occurrences  0.000000   
...                                                                  ...   
wound healing, spreading of epidermal cells (GO... p-value      0.002514   
zinc ion homeostasis (GO:0055069)                  Occurrences  0.000000   
                                                   p-value      1.000000   
zymogen activation (GO:0031638)                    Occurrences  0.000000   
                                                   p-value      1.000000   

                                                                          \
                                                                 Healthy   
'de novo' posttranslational protein folding (GO... Occurrences  0.030168   
                                                   p-value      0.000014   
5S class rRNA transcription from RNA polymerase... Occurrences  0.018994   
                                                   p-value      0.003888   
7-methylguanosine RNA capping (GO:0009452)         Occurrences  0.007821   
...                                                                  ...   
wound healing, spreading of epidermal cells (GO... p-value      0.002514   
zinc ion homeostasis (GO:0055069)                  Occurrences  0.000000   
                                                   p-value      1.000000   
zymogen activation (GO:0031638)                    Occurrences  0.000000   
                                                   p-value      1.000000   

                                                                               \
                                                                     IAV  RSV   
'de novo' posttranslational protein folding (GO... Occurrences  0.000000  0.0   
                                                   p-value      1.000000  1.0   
5S class rRNA transcription from RNA polymerase... Occurrences  0.000000  0.0   
                                                   p-value      1.000000  1.0   
7-methylguanosine RNA capping (GO:0009452)         Occurrences  0.002660  0.0   
...                                                                  ...  ...   
wound healing, spreading of epidermal cells (GO... p-value      0.002514  1.0   
zinc ion homeostasis (GO:0055069)                  Occurrences  0.000000  0.0   
                                                   p-value      1.000000  1.0   
zymogen activation (GO:0031638)                    Occurrences  0.000000  0.0   
                                                   p-value      1.000000  1.0   

                                                                           \
                                                               SARS-CoV-2   
'de novo' posttranslational protein folding (GO... Occurrences   0.056769   
                                                   p-value       0.000014   
5S class rRNA transcription from RNA polymerase... Occurrences   0.026201   
                                                   p-value       0.003888   
7-methylguanosine RNA capping (GO:0009452)         Occurrences   0.020378   
...                                                                   ...   
wound healing, spreading of epidermal cells (GO... p-value       0.002514   
zinc ion homeostasis (GO:0055069)                  Occurrences   0.000000   
                                                   p-value       1.000000   
zymogen activation (GO:0031638)                    Occurrences   0.000000   
                                                   p-value       1.000000   

                                  

In [37]:
#enriched_dataframe.xs(['Healthy', 'SARS-CoV-2'], axis=1, level=1, drop_level=False)
idx = pd.IndexSlice
enriched_dataframe.loc[idx[:, :], idx[['A549'], ['SARS-CoV-2']]]

A549
                                                               SARS-CoV-2
'de novo' posttranslational protein folding (GO... Occurrences   0.056769
                                                   p-value       0.000014
5S class rRNA transcription from RNA polymerase... Occurrences   0.026201
                                                   p-value       0.003888
7-methylguanosine RNA capping (GO:0009452)         Occurrences   0.020378
...                                                                   ...
wound healing, spreading of epidermal cells (GO... p-value       0.002514
zinc ion homeostasis (GO:0055069)                  Occurrences   0.000000
                                                   p-value       1.000000
zymogen activation (GO:0031638)                    Occurrences   0.000000
                                                   p-value       1.000000

[4096 rows x 1 columns]

In [38]:
enriched_dataframe['Calu3'].unstack()['SARS-CoV-2'].sort_values('p-value').head(25)

,Occurrences,p-value
gene expression (GO:0010467),0.145631,7.501327e-46
viral process (GO:0016032),0.077670,1.660156e-38
translation (GO:0006412),0.106796,1.660156e-38
nuclear-transcribed mRNA catabolic process (GO:0000956),0.106796,2.710695e-35
cytokine-mediated signaling pathway (GO:0019221),0.427184,2.581563e-33
cotranslational protein targeting to membrane (GO:0006613),0.116505,4.874873e-33
protein targeting to ER (GO:0045047),0.116505,2.929609e-32
mRNA processing (GO:0006397),0.145631,4.348842e-32
"nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)",0.106796,4.348842e-32
"RNA splicing, via transesterification reactions with bulged adenosine as nucleophile (GO:0000377)",0.145631,4.612540e-31


In [39]:
cell_types = {'NHBE', 'A549', 'Calu3', 'Biopsy'}

rank_processes = set()

for cell_type in cell_types:
    rank_processes.update(enriched_dataframe[cell_type].unstack()['SARS-CoV-2'].sort_values('p-value').head(25).index)

In [40]:
rank_processes

{'DNA metabolic process (GO:0006259)',
 'Golgi vesicle transport (GO:0048193)',
 'RNA export from nucleus (GO:0006405)',
 'RNA processing (GO:0006396)',
 'RNA splicing, via transesterification reactions with bulged adenosine as nucleophile (GO:0000377)',
 'SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
 'cellular macromolecule biosynthetic process (GO:0034645)',
 'cellular protein metabolic process (GO:0044267)',
 'cotranslational protein targeting to membrane (GO:0006613)',
 'cytokine-mediated signaling pathway (GO:0019221)',
 'gene expression (GO:0010467)',
 'inflammatory response (GO:0006954)',
 'mRNA export from nucleus (GO:0006406)',
 'mRNA processing (GO:0006397)',
 'mRNA splicing, via spliceosome (GO:0000398)',
 'mRNA transport (GO:0051028)',
 'mRNA-containing ribonucleoprotein complex export from nucleus (GO:0071427)',
 'mitotic cell cycle phase transition (GO:0044772)',
 'ncRNA processing (GO:0034470)',
 'neutrophil activation involved in immune res

In [41]:
enriched_dataframe.loc[idx[rank_processes, :], idx[:, ['Healthy', 'SARS-CoV-2']]].unstack()

A549                \
                                                       Healthy                 
                                                   Occurrences       p-value   
DNA metabolic process (GO:0006259)                    0.109497  1.269576e-22   
Golgi vesicle transport (GO:0048193)                  0.054749  6.968204e-27   
RNA export from nucleus (GO:0006405)                  0.039106  1.116166e-27   
RNA processing (GO:0006396)                           0.072626  5.631810e-35   
RNA splicing, via transesterification reactions...    0.094972  4.858980e-44   
SRP-dependent cotranslational protein targeting...    0.099441  9.747314e-32   
cellular macromolecule biosynthetic process (GO...    0.126257  9.268622e-47   
cellular protein metabolic process (GO:0044267)       0.112849  8.321845e-30   
cotranslational protein targeting to membrane (...    0.099441  2.531495e-34   
cytokine-mediated signaling pathway (GO:0019221)      0.753073  1.646143e-14   
gene expression (GO:0010467)                          0.132961  5.244679e-71   
inflammatory response (GO:0006954)                    0.555307  7.972553e-17   
mRNA export from nucleus (GO:0006406)                 0.042458  2.531391e-24   
mRNA processing (GO:0006397)                          0.108380  9.407986e-46   
mRNA splicing, via spliceosome (GO:0000398)           0.103911  6.450952e-46   
mRNA transport (GO:0051028)                           0.042458  1.153965e-24   
mRNA-containing ribonucleoprotein complex expor...    0.042458  3.174734e-24   
mitotic cell cycle phase transition (GO:0044772)      0.108380  4.691815e-20   
ncRNA processing (GO:0034470)                         0.124022  1.411431e-41   
neutrophil activation involved in immune respon...    0.089385  6.188743e-28   
neutrophil degranulation (GO:0043312)                 0.082682  1.920867e-28   
neutrophil mediated immunity (GO:0002446)             0.099441  6.188743e-28   
nuclear export (GO:0051168)                           0.036872  1.654895e-20   
nuclear-transcribed mRNA catabolic process (GO:...    0.111732  5.309261e-38   
nuclear-transcribed mRNA catabolic process, non...    0.098324  5.698886e-36   
peptide biosynthetic process (GO:0043043)             0.111732  3.644515e-30   
proteasomal protein catabolic process (GO:0010498)    0.075978  8.496438e-27   
proteasome-mediated ubiquitin-dependent protein...    0.102793  3.148284e-32   
protein modification by small protein conjugati...    0.092737  1.880907e-20   
protein polyubiquitination (GO:0000209)               0.073743  1.261987e-21   
protein targeting to ER (GO:0045047)                  0.099441  9.633068e-36   
protein ubiquitination (GO:0016567)                   0.089385  1.978981e-21   
rRNA metabolic process (GO:0016072)                   0.129609  7.032094e-38   
rRNA processing (GO:0006364)                          0.131844  4.173337e-38   
regulation of apoptotic process (GO:0042981)          0.060335  2.245640e-20   
regulation of mRNA stability (GO:0043488)             0.031285  5.995886e-28   
regulation of nucleic acid-templated transcript...    0.077095  3.591124e-08   
regulation of transcription from RNA polymerase...    0.087151  5.273073e-18   
regulation of translation (GO:0006417)                0.045810  1.246692e-34   
ribosome biogenesis (GO:0042254)                      0.153073  2.281380e-40   
transcription from RNA polymerase II promoter (...    0.088268  4.964519e-19   
translation (GO:0006412)                              0.112849  5.162406e-47   
ubiquitin-dependent protein catabolic process (...    0.109497  2.753293e-34   
viral gene expression (GO:0019080)                    0.098324  3.349751e-35   
viral process (GO:0016032)                            0.098324  9.461992e-47   
viral transcription (GO:0019083)                      0.098324  6.539993e-36   

                                                                              \
                                                    SARS-

|                                                                                                   |   ('A549', 'Healthy', 'Occurrences') |   ('A549', 'Healthy', 'p-value') |   ('A549', 'SARS-CoV-2', 'Occurrences') |   ('A549', 'SARS-CoV-2', 'p-value') |   ('Biopsy', 'Healthy', 'Occurrences') |   ('Biopsy', 'Healthy', 'p-value') |   ('Biopsy', 'SARS-CoV-2', 'Occurrences') |   ('Biopsy', 'SARS-CoV-2', 'p-value') |   ('Calu3', 'Healthy', 'Occurrences') |   ('Calu3', 'Healthy', 'p-value') |   ('Calu3', 'SARS-CoV-2', 'Occurrences') |   ('Calu3', 'SARS-CoV-2', 'p-value') |   ('NHBE', 'Healthy', 'Occurrences') |   ('NHBE', 'Healthy', 'p-value') |   ('NHBE', 'SARS-CoV-2', 'Occurrences') |   ('NHBE', 'SARS-CoV-2', 'p-value') |
|:--------------------------------------------------------------------------------------------------|-------------------------------------:|---------------------------------:|----------------------------------------:|------------------------------------:|---------------------------------------:|-----------------------------------:|------------------------------------------:|--------------------------------------:|--------------------------------------:|----------------------------------:|-----------------------------------------:|-------------------------------------:|-------------------------------------:|---------------------------------:|----------------------------------------:|------------------------------------:|
| DNA metabolic process (GO:0006259)                                                                |                                   98 |                      1.26958e-22 |                                      69 |                         1.26958e-22 |                                      4 |                        0.000206165 |                                        43 |                           1.21198e-21 |                                    11 |                       9.28125e-20 |                                        8 |                          1.06991e-16 |                                   32 |                      1.75923e-20 |                                       9 |                         5.73855e-07 |
| Golgi vesicle transport (GO:0048193)                                                              |                                   49 |                      6.9682e-27  |                                      55 |                         1.16731e-32 |                                      2 |                        3.99825e-23 |                                        20 |                           1.43768e-26 |                                     9 |                       5.06281e-26 |                                        4 |                          2.47511e-18 |                                    7 |                      7.36322e-08 |                                       4 |                         1.16731e-32 |
| RNA export from nucleus (GO:0006405)                                                              |                                   35 |                      1.11617e-27 |                                      50 |                         1.15543e-28 |                                      2 |                        4.05669e-23 |                                        14 |                           3.8944e-28  |                                     9 |                       7.01649e-26 |                                        4 |                          7.03378e-18 |                                    9 |                      8.9147e-10  |                                       4 |                         1.08789e-27 |
| RNA processing (GO:0006396)                                                                       |                                   65 |                      5.63181e-35 |                                      61 |                         5.63181e-35 |                                      6 |                        4.44316e-28 |                                        16 |                           1.01265e-24 |                                    15 |                       6.31143e-30 |                                       10 |                          1.28963e-17 |                                   23 |                      2.1743e-08  |                                      10 |                         6.28885e-31 |
| RNA splicing, via transesterification reactions with bulged adenosine as nucleophile (GO:0000377) |                                   85 |                      4.85898e-44 |                                      70 |                         7.49488e-45 |                                      7 |                        7.49488e-45 |                                        22 |                           4.90476e-38 |                                    18 |                       7.49488e-45 |                                       15 |                          4.61254e-31 |                                   39 |                      4.78274e-15 |                                      15 |                         9.30423e-45 |
| SRP-dependent cotranslational protein targeting to membrane (GO:0006614)                          |                                   89 |                      9.74731e-32 |                                      90 |                         9.74731e-32 |                                      6 |                        3.28067e-29 |                                         0 |                           1           |                                    16 |                       4.46604e-31 |                                       12 |                          1.11752e-30 |                                   25 |                      9.13492e-17 |                                      17 |                         5.26564e-28 |
| cellular macromolecule biosynthetic process (GO:0034645)                                          |                                  113 |                      9.26862e-47 |                                     107 |                         9.26862e-47 |                                      7 |                        4.43167e-27 |                                        16 |                           7.66366e-17 |                                    18 |                       9.26862e-47 |                                       11 |                          2.61793e-26 |                                   40 |                      5.7279e-10  |                                      19 |                         2.36741e-31 |
| cellular protein metabolic process (GO:0044267)                                                   |                                  101 |                      8.32185e-30 |                                     126 |                         1.08346e-29 |                                     15 |                        1.08346e-29 |                                         5 |                           7.38572e-07 |                                    23 |                       8.32185e-30 |                                       28 |                          5.60671e-26 |                                   29 |                      2.71047e-17 |                                      21 |                         1.25909e-27 |
| cotranslational protein targeting to membrane (GO:0006613)                                        |                                   89 |                      2.5315e-34  |                                      90 |                         2.5315e-34  |                                      6 |                        1.67102e-31 |                                         0 |                           1           |                                    16 |                       1.28118e-33 |                                       12 |                          4.87487e-33 |                                   25 |                      4.10496e-17 |                                      17 |                         1.70136e-30 |
| cytokine-mediated signaling pathway (GO:0019221)                                                  |                                  674 |                      1.64614e-14 |                                     247 |                         2.28928e-14 |                                     33 |                        2.58156e-33 |                                        30 |                           3.76444e-28 |                                    29 |                       3.42446e-21 |                                       44 |                          2.58156e-33 |                                   58 |                      9.2636e-08  |                                      26 |                         5.33936e-13 |
| gene expression (GO:0010467)                                                                      |                                  119 |                      5.24468e-71 |                                     110 |                         5.24468e-71 |                                      8 |                        1.44937e-62 |                                        14 |                           8.09428e-32 |                                    21 |                       1.31041e-68 |                                       15 |                          7.50133e-46 |                                   45 |                      1.90627e-10 |                                      22 |                         1.39042e-69 |
| inflammatory response (GO:0006954)                                                                |                                  497 |                      7.97255e-17 |                                     104 |                         8.80919e-06 |                                     15 |                        1.61475e-21 |                                        29 |                           1.61475e-21 |                                    19 |                       0.000181259 |                                       32 |                          9.40467e-06 |                                   82 |                      2.56751e-10 |                                      20 |                         3.45634e-09 |
| mRNA export from nucleus (GO:0006406)                                                             |                                   38 |                      2.53139e-24 |                                      50 |                         8.18476e-28 |                                      3 |                        8.98759e-23 |                                        18 |                           8.18476e-28 |                                     9 |                       3.00767e-24 |                                        5 |                          3.61794e-16 |                                   16 |                      2.1822e-11  |                                       4 |                         1.99252e-25 |
| mRNA processing (GO:0006397)                                                                      |                                   97 |                      9.40799e-46 |                                      72 |                         4.36225e-47 |                                      7 |                        1.45712e-46 |                                        25 |                           2.1262e-44  |                                    18 |                       1.45712e-46 |                                       15 |                          4.34884e-32 |                                   40 |                      8.05455e-18 |                                      15 |                         4.36225e-47 |
| mRNA splicing, via spliceosome (GO:0000398)                                                       |                                   93 |                      6.45095e-46 |                                      72 |                         4.83362e-47 |                                      7 |                        4.83362e-47 |                                        25 |                           9.36461e-40 |                                    18 |                       4.83362e-47 |                                       15 |                          1.25031e-30 |                                   39 |                      1.36227e-15 |                                      15 |                         2.77336e-46 |
| mRNA transport (GO:0051028)                                                                       |                                   38 |                      1.15397e-24 |                                      50 |                         1.91609e-26 |                                      3 |                        3.06278e-22 |                                        18 |                           1.91609e-26 |                                     9 |                       5.3231e-24  |                                        5 |                          7.03378e-18 |                                   16 |                      1.90627e-10 |                                       4 |                         6.59651e-25 |
| mRNA-containing ribonucleoprotein complex export from nucleus (GO:0071427)                        |                                   38 |                      3.17473e-24 |                                      50 |                         2.85729e-27 |                                      3 |                        6.96797e-22 |                                        18 |                           2.85729e-27 |                                     9 |                       3.21369e-24 |                                        5 |                          1.22417e-16 |                                   16 |                      1.67812e-10 |                                       4 |                         1.55747e-24 |
| mitotic cell cycle phase transition (GO:0044772)                                                  |                                   97 |                      4.69182e-20 |                                      72 |                         3.05655e-23 |                                      4 |                        6.17757e-09 |                                        38 |                           1.05924e-22 |                                    10 |                       1.63292e-18 |                                        5 |                          1.14811e-09 |                                   19 |                      7.20595e-10 |                                       8 |                         6.81177e-11 |
| ncRNA processing (GO:0034470)                                                                     |                                  111 |                      1.41143e-41 |                                     108 |                         1.41143e-41 |                                      7 |                        4.57834e-37 |                                        16 |                           6.32746e-12 |                                    19 |                       1.32745e-39 |                                       12 |                          7.18228e-22 |                                   34 |                      2.71317e-12 |                                      18 |                         1.37532e-38 |
| neutrophil activation involved in immune response (GO:0002283)                                    |                                   80 |                      6.18874e-28 |                                      78 |                         2.42928e-29 |                                     22 |                        2.42928e-29 |                                        14 |                           2.57769e-11 |                                    20 |                       1.47856e-29 |                                       26 |                          1.47856e-29 |                                   10 |                      7.36278e-08 |                                      16 |                         7.10886e-26 |
| neutrophil degranulation (GO:0043312)                                                             |                                   74 |                      1.92087e-28 |                                      78 |                         7.67272e-30 |                                     22 |                        7.67272e-30 |                                        14 |                           1.33796e-11 |                                    20 |                       4.81205e-30 |                                       26 |                          4.81205e-30 |                                   10 |                      5.80135e-08 |                                      16 |                         2.2203e-26  |
| neutrophil mediated immunity (GO:0002446)                                                         |                                   89 |                      6.18874e-28 |                                      93 |                         9.28059e-30 |                                     22 |                        9.28059e-30 |                                        14 |                           2.52722e-11 |                                    23 |                       1.74622e-30 |                                       29 |                          1.74622e-30 |                                   10 |                      3.95449e-08 |                                      16 |                         8.28863e-26 |
| nuclear export (GO:0051168)                                                                       |                                   33 |                      1.6549e-20  |                                      50 |                         1.89945e-21 |                                      2 |                        1.77619e-19 |                                        14 |                           1.89945e-21 |                                     9 |                       3.00048e-19 |                                        4 |                          3.40317e-14 |                                    5 |                      1.44284e-07 |                                       4 |                         6.06419e-21 |
| nuclear-transcribed mRNA catabolic process (GO:0000956)                                           |                                  100 |                      5.30926e-38 |                                      99 |                         5.30926e-38 |                                      7 |                        9.21182e-35 |                                         4 |                           0.000358209 |                                    16 |                       2.7107e-35  |                                       11 |                          2.7107e-35  |                                   32 |                      1.85635e-15 |                                      19 |                         2.26178e-36 |
| nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)                  |                                   88 |                      5.69889e-36 |                                      90 |                         5.69889e-36 |                                      6 |                        2.36346e-30 |                                         0 |                           1           |                                    16 |                       4.34884e-32 |                                       11 |                          4.34884e-32 |                                   23 |                      1.08551e-15 |                                      15 |                         2.36741e-31 |
| peptide biosynthetic process (GO:0043043)                                                         |                                  100 |                      3.64451e-30 |                                      95 |                         9.10857e-29 |                                      6 |                        1.79589e-28 |                                         1 |                           0.00761285  |                                    16 |                       3.64451e-30 |                                       11 |                          1.47073e-24 |                                   32 |                      4.18859e-13 |                                      19 |                         1.37685e-27 |
| proteasomal protein catabolic process (GO:0010498)                                                |                                   68 |                      8.49644e-27 |                                      67 |                         8.49644e-27 |                                      2 |                        7.0868e-24  |                                        24 |                           1.10262e-21 |                                     9 |                       9.44929e-26 |                                        4 |                          2.1454e-24  |                                   19 |                      3.45707e-07 |                                       5 |                         2.77963e-24 |
| proteasome-mediated ubiquitin-dependent protein catabolic process (GO:0043161)                    |                                   92 |                      3.14828e-32 |                                      78 |                         3.14828e-32 |                                      2 |                        1.77787e-28 |                                        31 |                           1.1125e-26  |                                    10 |                       1.94351e-31 |                                        4 |                          1.618e-23   |                                   19 |                      2.14749e-09 |                                       5 |                         3.84067e-28 |
| protein modification by small protein conjugation (GO:0032446)                                    |                                   83 |                      1.88091e-20 |                                      63 |                         2.83275e-26 |                                      2 |                        4.77469e-23 |                                        33 |                           2.83275e-26 |                                    10 |                       8.35601e-21 |                                        4 |                          1.91633e-14 |                                   23 |                      4.76695e-14 |                                       4 |                         9.33756e-24 |
| protein polyubiquitination (GO:0000209)                                                           |                                   66 |                      1.26199e-21 |                                      61 |                         3.04458e-26 |                                      4 |                        1.58321e-24 |                                        25 |                           9.88752e-22 |                                    13 |                       1.58321e-24 |                                        7 |                          2.49279e-16 |                                   19 |                      6.18825e-09 |                                       7 |                         1.0173e-24  |
| protein targeting to ER (GO:0045047)                                                              |                                   89 |                      9.63307e-36 |                                      90 |                         9.63307e-36 |                                      6 |                        7.25047e-33 |                                         0 |                           1           |                                    16 |                       4.82677e-35 |                                       12 |                          2.92961e-32 |                                   25 |                      2.36211e-18 |                                      17 |                         9.79939e-32 |
| protein ubiquitination (GO:0016567)                                                               |                                   80 |                      1.97898e-21 |                                      68 |                         3.73116e-25 |                                      4 |                        2.42697e-22 |                                        37 |                           5.21366e-24 |                                    13 |                       2.42697e-22 |                                        8 |                          1.65581e-16 |                                   25 |                      2.50916e-11 |                                      11 |                         2.85791e-24 |
| rRNA metabolic process (GO:0016072)                                                               |                                  116 |                      7.03209e-38 |                                     112 |                         7.03209e-38 |                                      7 |                        5.33316e-36 |                                        12 |                           6.24901e-09 |                                    21 |                       5.33316e-36 |                                       14 |                          3.67369e-23 |                                   37 |                      5.21409e-12 |                                      20 |                         5.79455e-36 |
| rRNA processing (GO:0006364)                                                                      |                                  118 |                      4.17334e-38 |                                     110 |                         4.17334e-38 |                                      7 |                        5.05604e-37 |                                        13 |                           1.08335e-09 |                                    21 |                       5.05604e-37 |                                       14 |                          2.05815e-23 |                                   41 |                      1.64876e-12 |                                      20 |                         3.72695e-36 |
| regulation of apoptotic process (GO:0042981)                                                      |                                   54 |                      2.24564e-20 |                                      96 |                         1.02067e-22 |                                     15 |                        9.02299e-19 |                                        12 |                           6.0434e-12  |                                    20 |                       1.68067e-24 |                                       24 |                          1.68067e-24 |                                   11 |                      1.50912e-09 |                                      17 |                         1.02067e-22 |
| regulation of mRNA stability (GO:0043488)                                                         |                                   28 |                      5.99589e-28 |                                      50 |                         5.99589e-28 |                                      7 |                        1.527e-26   |                                         6 |                           8.53035e-14 |                                    15 |                       1.527e-26   |                                        7 |                          9.59407e-26 |                                    3 |                      0.0033952   |                                      10 |                         2.00551e-26 |
| regulation of nucleic acid-templated transcription (GO:1903506)                                   |                                   69 |                      3.59112e-08 |                                      24 |                         5.39242e-07 |                                      0 |                        1           |                                        22 |                           5.97943e-20 |                                     4 |                       0.000199887 |                                        5 |                          0.000199887 |                                   27 |                      5.97943e-20 |                                       9 |                         0.00038458  |
| regulation of transcription from RNA polymerase II promoter (GO:0006357)                          |                                   78 |                      5.27307e-18 |                                      87 |                         8.91675e-22 |                                     10 |                        3.05561e-21 |                                        37 |                           6.70936e-20 |                                    19 |                       3.05561e-21 |                                       24 |                          1.35711e-18 |                                   23 |                      5.44718e-12 |                                      11 |                         8.65472e-21 |
| regulation of translation (GO:0006417)                                                            |                                   41 |                      1.24669e-34 |                                      57 |                         9.13153e-35 |                                      2 |                        5.30446e-31 |                                        13 |                           7.20495e-26 |                                     9 |                       1.24669e-34 |                                        5 |                          1.71159e-24 |                                    7 |                      1.97864e-06 |                                       5 |                         1.10261e-31 |
| ribosome biogenesis (GO:0042254)                                                                  |                                  137 |                      2.28138e-40 |                                     120 |                         2.59951e-41 |                                      9 |                        6.65154e-39 |                                        17 |                           8.01216e-14 |                                    24 |                       6.65154e-39 |                                       17 |                          4.87225e-22 |                                   48 |                      2.50556e-12 |                                      23 |                         8.09548e-39 |
| transcription from RNA polymerase II promoter (GO:0006366)                                        |                                   79 |                      4.96452e-19 |                                      63 |                         7.14513e-24 |                                      4 |                        2.03234e-23 |                                        32 |                           1.86159e-23 |                                    13 |                       1.86159e-23 |                                        7 |                          2.19275e-15 |                                   23 |                      1.4819e-16  |                                       7 |                         7.14513e-24 |
| translation (GO:0006412)                                                                          |                                  101 |                      5.16241e-47 |                                     103 |                         7.08123e-46 |                                      7 |                        8.77169e-43 |                                         8 |                           1.94154e-11 |                                    20 |                       5.16241e-47 |                                       11 |                          1.66016e-38 |                                   32 |                      5.69215e-21 |                                      19 |                         2.14661e-44 |
| ubiquitin-dependent protein catabolic process (GO:0006511)                                        |                                   98 |                      2.75329e-34 |                                      86 |                         2.75329e-34 |                                      6 |                        3.13871e-31 |                                        34 |                           6.47819e-27 |                                    15 |                       1.04261e-33 |                                       11 |                          6.43713e-27 |                                   28 |                      2.14749e-09 |                                       9 |                         2.67759e-33 |
| viral gene expression (GO:0019080)                                                                |                                   88 |                      3.34975e-35 |                                      88 |                         3.34975e-35 |                                      6 |                        2.69948e-29 |                                         0 |                           1           |                                    15 |                       2.01767e-33 |                                       11 |                          3.91074e-28 |                                   25 |                      3.66167e-14 |                                      16 |                         3.31897e-31 |
| viral process (GO:0016032)                                                                        |                                   88 |                      9.46199e-47 |                                      94 |                         9.46199e-47 |                                      7 |                        2.5091e-42  |                                         5 |                           1.42961e-08 |                                    15 |                       1.19157e-45 |                                        8 |                          1.66016e-38 |                                   25 |                      2.47996e-16 |                                      17 |                         2.20416e-41 |
| viral transcription (GO:0019083)                                                                  |                                   88 |                      6.53999e-36 |                                      88 |                         6.53999e-36 |                                      6 |                        5.78956e-30 |                                         0 |                           1           |                                    15 |                       3.48035e-35 |                                       11 |                          6.53912e-30 |                                   25 |                      1.27285e-15 |                                      16 |                         7.67572e-32 |
​